In [1]:
import os, sys
sys.path.insert(1, os.path.join(os.getcwd()  , '../..'))

In [2]:
# imports
import random
from core import * 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [3]:
possible_items = [
    TwoDimensionalItem(300, 400),
    TwoDimensionalItem(600, 300),
    TwoDimensionalItem(600, 400),
    TwoDimensionalItem(300, 200),
]

vehicle = Vehicle(
    compartments=[
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 300),
        TwoDimensionalCompartment(800, 600),
        TwoDimensionalCompartment(800, 600),
    ]
)

# Define some problems

# Problem 1
nb_items_1 = 15
V_1 = list(range(0, nb_items_1 * 2 + 1))
random.seed(0)
items_1 = random.choices(possible_items, k=nb_items_1)
random.seed(0)
distance_matrix_1 = [
    [random.randint(1, 100) if i != 0 and j!=0 and i != j else 0 for i in V_1]
    for j in V_1
]
problem_1 = TwoDimensionalProblem(items_1, vehicle, distance_matrix_1)

# Problem 2
nb_items_2 = 15
V_2 = list(range(0, nb_items_2 * 2 + 1))
random.seed(100)
items_2 = random.choices(possible_items, k=nb_items_2)
random.seed(100)
distance_matrix_2 = [
    [random.randint(1, 100) if i != 0 and j!=0 and i != j else 0 for i in V_2]
    for j in V_2
]
problem_2 = TwoDimensionalProblem(items_2, vehicle, distance_matrix_2)

In [4]:
# Finding inital solution with gurobi
# Initial solutions for the problem by running the gurobi solver
initial_solution_time_limit = 2 # seconds
# Problem 1
modelled_problem_1 = ModelledTwoDimensionalProblem(items_1, vehicle)
modelled_problem_1.create_model()
modelled_problem_1.apply_constraints()
modelled_problem_1.set_model_objective()
modelled_problem_1.solve(time_limit=initial_solution_time_limit)
initial_solution_1 = modelled_problem_1.extract_solution()

# Problem 2
modelled_problem_2 = ModelledTwoDimensionalProblem(items_2, vehicle)
modelled_problem_2.create_model()
modelled_problem_2.apply_constraints()
modelled_problem_2.set_model_objective()
modelled_problem_2.solve(time_limit=initial_solution_time_limit)
initial_solution_2 = modelled_problem_2.extract_solution()

initial_solutions_dict = {
    problem_1: initial_solution_1,
    problem_2: initial_solution_2,
}

solvers = dict()

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-05-14
Set parameter TimeLimit to value 2
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 12819 rows, 1469 columns and 61479 nonzeros
Model fingerprint: 0xf772a26f
Variable types: 0 continuous, 1469 integer (1252 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 8e+02]
Presolve removed 1255 rows and 298 columns
Presolve time: 0.19s
Presolved: 11564 rows, 1171 columns, 43492 nonzeros
Variable types: 0 continuous, 1171 integer (961 binary)
Found heuristic solution: objective 1159.0000000

Root relaxation: objective 1.397000e+02, 522 iterations, 0.06 seconds (0.05 work units)

    Nodes    |    Curre

In [5]:
# Solve using LNS with biased highest cost destroy
time_limit = 100
solver = LNS(problem_1, initial_solution_1, time_limit=time_limit)
solver.set_destroy_strategy(DestroyStrategy.BIASED_RANDOM_HIGHEST_COST)
solver.set_repair_strategy(RepairStrategy.RANDOM_SINGLE_ORDER_LEAST_COST)
bhc_destroy_slc_insert_solver = solver 
solvers["bhc_slc"] = solver


# Same but with parallel least cost insert, defualt number of processes cpus - 1
solver = PLNS(problem_1, initial_solution_1, time_limit=time_limit)
solver.set_destroy_strategy(DestroyStrategy.BIASED_RANDOM_HIGHEST_COST)
bhc_destroy_plc_insert_solver = solver 
solvers["bhc_plc"] = solver

Parallel LNS initialized with 11 processes


In [6]:
# run the search
bhc_destroy_slc_insert_solver.search()

Terminating LNS in 457 iterations
Total time: 100.15 Best objective: 256 found in 258 iterations in 54.38 s



Solution(
	order=0, 8, 6, 12, 10, 21, 25, 9, 3, 14, 24, 11, 13, 26, 2, 27, 5, 20, 17, 7, 23, 4, 15, 28, 22, 19, 1, 30, 18, 16, 29, 
	step: 0 vertex: 0 	0, 0, 0, 0, 0, 0
	step: 1 vertex: 8 	0, 0, 0, 0, 300, 0
	step: 2 vertex: 6 	600, 0, 0, 0, 300, 0
	step: 3 vertex: 12 	600, 0, 0, 0, 700, 0
	step: 4 vertex: 10 	600, 0, 0, 0, 700, 400
	step: 5 vertex: 21 	0, 0, 0, 0, 700, 400
	step: 6 vertex: 25 	0, 0, 0, 0, 700, 0
	step: 7 vertex: 9 	600, 0, 0, 0, 700, 0
	step: 8 vertex: 3 	600, 0, 0, 600, 700, 0
	step: 9 vertex: 14 	600, 200, 0, 600, 700, 0
	step: 10 vertex: 24 	0, 200, 0, 600, 700, 0
	step: 11 vertex: 11 	200, 200, 0, 600, 700, 0
	step: 12 vertex: 13 	200, 200, 600, 600, 700, 0
	step: 13 vertex: 26 	0, 200, 600, 600, 700, 0
	step: 14 vertex: 2 	0, 400, 600, 600, 700, 0
	step: 15 vertex: 27 	0, 400, 600, 600, 300, 0
	step: 16 vertex: 5 	0, 400, 600, 600, 700, 0
	step: 17 vertex: 20 	0, 400, 600, 600, 300, 0
	step: 18 vertex: 17 	0, 200, 600, 600, 300, 0
	step: 19 vertex: 7 	200, 200, 6

In [7]:
# run the search
bhc_destroy_plc_insert_solver.search()

Parallel LNS timed out, not all insertion orders could be tried.
Terminating LNS in 258 iterations
Total time: 100.00 Best objective: 255 found in 23 iterations in 9.38 s



Solution(
	order=0, 8, 4, 12, 10, 11, 13, 26, 2, 27, 5, 20, 17, 7, 25, 6, 22, 19, 28, 15, 23, 9, 3, 14, 24, 21, 1, 30, 18, 16, 29, 
	step: 0 vertex: 0 	0, 0, 0, 0, 0, 0
	step: 1 vertex: 8 	600, 0, 0, 0, 0, 0
	step: 2 vertex: 4 	600, 0, 600, 0, 0, 0
	step: 3 vertex: 12 	600, 0, 600, 0, 0, 400
	step: 4 vertex: 10 	600, 0, 600, 0, 400, 400
	step: 5 vertex: 11 	800, 0, 600, 0, 400, 400
	step: 6 vertex: 13 	800, 0, 600, 600, 400, 400
	step: 7 vertex: 26 	600, 0, 600, 600, 400, 400
	step: 8 vertex: 2 	800, 0, 600, 600, 400, 400
	step: 9 vertex: 27 	800, 0, 600, 600, 400, 0
	step: 10 vertex: 5 	800, 0, 600, 600, 800, 0
	step: 11 vertex: 20 	800, 0, 600, 600, 400, 0
	step: 12 vertex: 17 	600, 0, 600, 600, 400, 0
	step: 13 vertex: 7 	800, 0, 600, 600, 400, 0
	step: 14 vertex: 25 	800, 0, 600, 600, 0, 0
	step: 15 vertex: 6 	800, 600, 600, 600, 0, 0
	step: 16 vertex: 22 	600, 600, 600, 600, 0, 0
	step: 17 vertex: 19 	600, 600, 0, 600, 0, 0
	step: 18 vertex: 28 	600, 600, 0, 0, 0, 0
	step: 19 vert

In [8]:
def get_solver_stats(solver):
    objectives = [sol.objective for sol in solver.solutions_cache]
    iterations = [sol.iteration for sol in solver.solutions_cache]
    time_found = [
        "{:.2f}".format(sol.time_to_find) for sol in solver.solutions_cache
    ]

    improving_solutions = []
    best_obj = solver.solutions_cache[0].objective
    for i, sol in enumerate(solver.solutions_cache):
        if sol.objective < best_obj:
            best_obj = sol.objective
            improving_solutions.append(sol)

    improving_iterations = [sol.iteration for sol in improving_solutions]
    improving_objectives = [sol.objective for sol in improving_solutions]
    improving_times = [
        "{:.2f}".format(sol.time_to_find) for sol in improving_solutions
    ]

    stats = dict(
        objectives=objectives,
        iterations=iterations,
        times_found=time_found,
        improving_iterations=improving_iterations,
        improving_objectives=improving_objectives,
        improving_times=improving_times,
    )

    return stats

In [10]:
fig = make_subplots(
        rows=len(solvers),
        cols=2,
        shared_yaxes=True,
        shared_xaxes=True,
    )

r = 1
for name, solver in solvers.items():
    stats = get_solver_stats(solver)
    
    # transpartent light blue 
    fig.add_trace(go.Scatter(x=stats["iterations"],
                            y=stats["objectives"],
                            name=name+" all iterations",
                            opacity=0.5,
                            marker_color="#64a0c8",
                            ),
                row=r,
                col=1)
    
    # blue
    fig.add_trace(go.Scatter(x=stats["improving_iterations"],
                            y=stats["improving_objectives"],
                            name=name+" improving iterations",
                            marker_color="#005293",
                            ),
                row=r,
                col=1)

    # transparent light green
    fig.add_trace(go.Scatter(x=stats["times_found"],
                            y=stats["objectives"],
                            name=name+" all times",
                            opacity=0.5,
                            marker_color="#b6d7a8",
                            ),
                row=r,
                col=2)
    
    # green
    fig.add_trace(go.Scatter(x=stats["improving_times"],
                            y=stats["improving_objectives"],
                            name=name+" improving times",
                            marker_color="#38761d",
                            ),
                row=r,
                col=2)

    fig.update_xaxes(title_text=name + " iteration", row=r, col=1)
    fig.update_xaxes(title_text=name + " time", row=r, col=2)
    fig.update_yaxes(title_text="objective", row=r, col=1)

    r += 1

fig.update_layout(height=1000, width=1600, title_text="Objective Value as the LNS Progresses")
    

fig.show()